In [1]:
%load_ext jupyter_black

import os
from datetime import datetime
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import glob
from shapely.geometry import Point
from datetime import timedelta
import warnings


In [2]:
warnings.filterwarnings("ignore")
AA_DATA_DIR = os.getenv("AA_DATA_DIR")
save_dir = Path(AA_DATA_DIR) / "public" / "exploration" / "moz" / "ecmwf_hindcast"
ADMS = ["Sofala", "Inhambane", "Nampula", "Zambezia"]
adm1_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "raw"
    / "moz"
    / "cod_ab"
    / "moz_admbnda_adm1_ine_20190607.shp"
)
gdf_adm1 = gpd.read_file(adm1_path)
gdf_sel = gdf_adm1[gdf_adm1.ADM1_PT.isin(ADMS)]

In [3]:
def categorize_cyclone(wind_speed):
    if wind_speed > 115:
        return "Very Intense Tropical Cyclone"
    elif wind_speed >= 90:
        return "Intense Tropical Cyclone"
    elif wind_speed >= 64:
        return "Tropical Cyclone"
    elif wind_speed >= 48:
        return "Severe Tropical Storm"
    elif wind_speed >= 34:
        return "Moderate Tropical Storm"
    elif wind_speed >= 28:
        return "Tropical Depression"
    else:
        return "Tropical Disturbance"

In [5]:
moz_cyclones = [
    Path(f).stem.replace("_all", "").upper()
    for f in glob.glob(str(Path(save_dir) / "csv" / "*_all.csv"))
]
cyclone_speed = []
for cyclone_file_path in glob.glob(str(save_dir / "csv/*_all.csv")):
    cyclone_name = Path(cyclone_file_path).stem.split("_")[0]
    print(f"Processing file: {cyclone_file_path}")
    cyclone_file = pd.read_csv(cyclone_file_path)
    cyclone_file["time"] = pd.to_datetime(cyclone_file["time"])

    cyclone_df = (
        cyclone_file[["time", "speed", "lat", "lon", "lead_time", "forecast_time"]]
        .groupby(["time", "forecast_time"])
        .median()
        .reset_index()
    )
    cyclone_df["lat"] = cyclone_df["lat"].apply(lambda x: -x if x > 0 else x)

    cyclone_df["speed_knots"] = cyclone_df["speed"] * 1.94384
    cyclone_df["storm_category"] = cyclone_df["speed_knots"].apply(categorize_cyclone)
    cyc_ls = []
    for ts, ts_df in cyclone_df.groupby("forecast_time"):
        lt_0_df = gpd.GeoDataFrame(
            ts_df, geometry=gpd.points_from_xy(ts_df.lon, ts_df.lat), crs="EPSG:4326"
        )
        cyc_sjoin = gpd.sjoin(lt_0_df, gdf_sel, how="left", predicate="intersects")
        lt_0_df["within_land"] = cyc_sjoin["index_right"].notna()
        lt_0_df["point_location"] = np.where(
            lt_0_df["within_land"], "Within", "Outside"
        )

        first_landfall = (
            lt_0_df[lt_0_df["within_land"]].index[0]
            if not lt_0_df[lt_0_df["within_land"]].empty
            else None
        )
        if first_landfall is None:
            continue
        lt_0_df.loc[lt_0_df.index == first_landfall, "point_location"] = "Landfall"
        landfall_time = pd.to_datetime(
            lt_0_df[lt_0_df["point_location"] == "Landfall"]["time"].values[0]
        )
        lt_0_df["time_to_landfall"] = (
            landfall_time - pd.to_datetime(lt_0_df["forecast_time"])
        ).dt.total_seconds() / 3600
        lt_0_df["time_from_landfall"] = (
            lt_0_df["lead_time"] - lt_0_df["time_to_landfall"]
        )
        # lt_0_df = lt_0_df[(lt_0_df["time_to_landfall"] <= 12)]
        cyc_df = lt_0_df  # [(lt_0_df["point_location"] != "Outside") | ((lt_0_df["time_from_landfall"] >= -12) & (lt_0_df["time_from_landfall"] <= 12))]
        cyc_ls.append(cyc_df)
    if len(cyc_ls) > 0:
        cyc_df = pd.concat(cyc_ls)
        cyc_df["storm"] = cyclone_name.upper()
        cyclone_speed.append(cyc_df)

cyclone_out = pd.concat(cyclone_speed)

Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\favio_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\jokwe_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\izilda_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\dando_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\irina_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\haruna_all.

In [8]:
cyclone_speed = []
for cyclone_file_path in glob.glob(str(save_dir / "csv/*_all.csv")):
    cyclone_name = Path(cyclone_file_path).stem.split("_")[0]
    print(f"Processing file: {cyclone_file_path}")
    cyclone_file = pd.read_csv(cyclone_file_path)
    cyclone_file["time"] = pd.to_datetime(cyclone_file["time"])

    cyclone_df = (
        cyclone_file[["time", "speed", "lat", "lon", "lead_time", "forecast_time"]]
        .groupby(["time", "forecast_time"])
        .median()
        .reset_index()
    )
    cyclone_df["lat"] = cyclone_df["lat"].apply(lambda x: -x if x > 0 else x)

    cyclone_df["speed_knots"] = cyclone_df["speed"] * 1.94384
    cyclone_df["storm_category"] = cyclone_df["speed_knots"].apply(categorize_cyclone)
    cyc_ls = []
    lt_0_df = gpd.GeoDataFrame(
        cyclone_df,
        geometry=gpd.points_from_xy(cyclone_df.lon, cyclone_df.lat),
        crs="EPSG:4326",
    )
    cyc_sjoin = gpd.sjoin(lt_0_df, gdf_sel, how="left", predicate="intersects")
    lt_0_df["within_land"] = cyc_sjoin["index_right"].notna()
    lt_0_df["point_location"] = np.where(lt_0_df["within_land"], "Within", "Outside")

    first_landfall = (
        lt_0_df[lt_0_df["within_land"]].index[0]
        if not lt_0_df[lt_0_df["within_land"]].empty
        else None
    )
    if first_landfall is None:
        continue
    lt_0_df.loc[lt_0_df.index == first_landfall, "point_location"] = "Landfall"
    landfall_time = pd.to_datetime(
        lt_0_df[lt_0_df["point_location"] == "Landfall"]["time"].values[0]
    )
    lt_0_df["time_to_landfall"] = (
        landfall_time - pd.to_datetime(lt_0_df["forecast_time"])
    ).dt.total_seconds() / 3600
    lt_0_df["time_from_landfall"] = lt_0_df["lead_time"] - lt_0_df["time_to_landfall"]
    # lt_0_df = lt_0_df[(lt_0_df["time_to_landfall"] <= 12)]
    cyc_df = lt_0_df  # [(lt_0_df["point_location"] != "Outside") | ((lt_0_df["time_from_landfall"] >= -12) & (lt_0_df["time_from_landfall"] <= 12))]
    cyc_ls.append(cyc_df)
    if len(cyc_ls) > 0:
        cyc_df = pd.concat(cyc_ls)
        cyc_df["storm"] = cyclone_name.upper()
        cyclone_speed.append(cyc_df)

cyclone_out = pd.concat(cyclone_speed)

Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\favio_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\jokwe_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\izilda_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\dando_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\irina_all.csv
Processing file: G:\Shared drives\Predictive Analytics\CERF Anticipatory Action\General - All AA projects\Data\public\exploration\moz\ecmwf_hindcast\csv\haruna_all.

In [9]:
cyclone_out.to_csv(
    "C:/Users/pauni/Desktop/Work/OCHA/Mozambique/cyclone_landfall_speed.csv",
    index=False,
)